# Exam Example 4 - full simulation

## Part 1 - MCQ

1. The correct answer is b) 3 bock. Indeed, hdfs blocks can only contain part of one file, hence since `log2.txt` is bigger than 1 block size, 2 blocks are needed to store it (it cannot be stored together with the other file).
2. a0 Cache `logsRDD`, since a count action is done on it and another is done on its descendant `distinctURLs`.

## Part 2 - Exercises


### Exercise 1

In [38]:
# With RDDs:

import sys

#input_path = sys.argv[1]
input_path = '/data/students/bigdata_internet/exam_examples_data/example4_data/Prices.txt'

in_RDD = sc.textFile(input_path).map(lambda line: line.split(','))

def simplify(l1, l2):
    
    dateHour1 = str(l1[0]) + ',' + str(l1[1])
    dateHour2 = str(l2[0]) + ',' + str(l2[1])
    
    # Lowest price
    if float(l1[3]) > float(l2[3]):
        return l2
    elif float(l1[3]) < float(l2[3]): 
        return l1
    else:
        if dateHour1 > dateHour2:
            return l1
        else:
            return l2

out_better = in_RDD.filter(lambda el: int(el[0].split('/')[0]) == 2014 and str(el[2]) == 'TSLA')\
        .reduce(simplify)

out_def = out_better[0] + ',' + out_better[1] + ',' + out_better[3]

print(out_def)

# 2014/05/21,15:16,5.32

2014/05/21,15:16,5.32


In [5]:
filt_RDD.collect()

[['2014/05/21', '15:16', '55.32'],
 ['2014/05/21', '15:12', '45.32'],
 ['2014/05/21', '15:08', '45.32'],
 ['2014/05/21', '15:04', '45.32'],
 ['2014/05/21', '15:00', '45.32'],
 ['2014/05/21', '15:10', '15.32'],
 ['2014/05/21', '15:06', '15.32'],
 ['2014/05/21', '15:02', '15.32'],
 ['2014/05/21', '15:16', '5.32'],
 ['2014/05/21', '15:14', '5.32']]

In [ ]:
# With DFs

spark.udf.register('getYear', lambda date: int(date.split('/')[0]))
spark.udf.register('getMonth', lambda date: int(date.split('/')[1]))
spark.udf.register('getDay', lambda date: int(date.split('/')[2]))
spark.udf.register('getHour', lambda h: int(h.split(':')[0]))
spark.udf.register('getMin', lambda h: int(h.split(':')[1]))

in_DF_1 = spark.read.load(input_path, format='csv', header=False, inferSchema=True).asDF('date', 'time', 'stockID', 'price')

in_DF_1.createOrReplaceTempView('input_tab')

out_1_DF = in_DF_1.filter('stockID == "TSLA" AND getYear(date) == 2014')\
    .select('date', 'time', 'price').sort('price', 'getMonth(date)', 'getDay(date)', 'getHour(time)', 'getMin(time)')

# NO NEED TO SORT THE WHOLE DF... 

In [36]:
'ab' > 'aa'

True

### Exercise 2

In [ ]:
!hdfs dfs -rm -r /user/s315054/exam_sim/example4/ex2a

In [18]:
# With DFs

# Part a
import sys

# input_f = sys.argv[1] 
# out_a = sys.argv[2]
# out_b = sys.argv[3]

#input_f = '/data/students/bigdata_internet/exam_examples_data/example4_data/Prices.txt'
input_f = '/user/s315054/Prices.txt' 
out_a = '/user/s315054/exam_sim/example4/ex2a'
out_b = '/user/s315054/exam_sim/example4/ex2b'

in_DF = spark.read.load(input_f, format='csv', header=False).toDF('date', 'time', 'stockID', 'price')

spark.udf.register('getYear', lambda date: int(date.split('/')[0]))
spark.udf.register('getMonth', lambda date: int(date.split('/')[1]))
spark.udf.register('getDay', lambda date: int(date.split('/')[2]))

in_DF.createOrReplaceTempView('input_tab')

# Isolate 2014 records, group them per date and id and take max and min prices for each
filt_better_DF = spark.sql("""
        SELECT date, stockID, max(price), min(price)
        FROM input_tab
        WHERE getYear(date) == 2014
        GROUP BY date, stockID
""").withColumnRenamed('max(price)', 'max_price').withColumnRenamed('min(price)', 'min_price')

# Only keep the ones in which the daily price variation is < 5
# Then Group by ID and count the ones remaining for each stock
less_5_daily_DF = filt_better_DF.filter("max_price - min_price < 5")\
    .groupBy('stockID').agg({'*': 'count'}).withColumnRenamed('count(1)', 'n_variation_less_5')

# Produce output:
less_5_daily_DF.write.csv(out_a, header=False, sep=',')


23/01/30 16:03:59 WARN analysis.SimpleFunctionRegistry: The function getyear replaced a previously registered function.
23/01/30 16:03:59 WARN analysis.SimpleFunctionRegistry: The function getmonth replaced a previously registered function.
23/01/30 16:03:59 WARN analysis.SimpleFunctionRegistry: The function getday replaced a previously registered function.


In [19]:
less_5_daily_DF.show()

+-------+--------+
|stockID|count(1)|
+-------+--------+
|   GOOG|       2|
|    FCA|       1|
+-------+--------+



In [28]:
# Part b:
import datetime

dpv_DF = filt_better_DF.selectExpr('date', 'stockID', 'max_price - min_price AS dpv')

#########
def getPrevDay(date):
    date1 = datetime.datetime.strptime(date, '%Y/%m/%d')
    yesterday = date1 - datetime.timedelta(1)
    return yesterday.strftime('%Y/%m/%d')

spark.udf.register('getPrevDay', getPrevDay)

with_yesterday_DF = dpv_DF.selectExpr('date', 'stockID', 'dpv', 'getPrevDay(date) AS yesterday')

today_and_yesterday_DF = with_yesterday_DF.join(dpv_DF.withColumnRenamed('date', 'yesterday').withColumnRenamed('dpv', 'dpv_yesterday'), ['yesterday', 'stockID'])

unstable_pairs_DF = today_and_yesterday_DF.filter("abs(dpv - dpv_yesterday) > 1")

unstable_pairs_DF.selectExpr('stockID', 'yesterday AS first_date').write.csv(out_b, header=False)

23/01/30 16:26:30 WARN analysis.SimpleFunctionRegistry: The function getprevday replaced a previously registered function.


In [24]:
getPrevDay('2023/01/30')

'2023/01/29'

In [29]:
unstable_pairs_DF.show()

+----------+-------+----------+---+-------------+
| yesterday|stockID|      date|dpv|dpv_yesterday|
+----------+-------+----------+---+-------------+
|2014/05/21|   GOOG|2014/05/22|2.0|         40.0|
+----------+-------+----------+---+-------------+



# Cazzeggio

In [56]:
spark.udf.register('makelst', lambda d: d.split('/'))

withList = in_DF.selectExpr('makelst(date) AS date_list', 'time', 'stockID', 'price')

#withList.select('date_lst.1').show()
print(withList.collect()[0].asDict()['date_list'][0])

[


23/01/30 18:58:33 WARN analysis.SimpleFunctionRegistry: The function makelst replaced a previously registered function.


In [53]:
withList.select('time', 'time').show()

+-----+-----+
| time| time|
+-----+-----+
|15:00|15:00|
|15:02|15:02|
|15:04|15:04|
|15:06|15:06|
|15:08|15:08|
|15:10|15:10|
|15:12|15:12|
|15:14|15:14|
|15:16|15:16|
|15:16|15:16|
|15:00|15:00|
|15:02|15:02|
|15:04|15:04|
|15:06|15:06|
|15:08|15:08|
|15:10|15:10|
|15:12|15:12|
|15:14|15:14|
|15:16|15:16|
|15:16|15:16|
+-----+-----+
only showing top 20 rows

